## Storing and Querying Embeddings with AlloyDB for PostgreSQL

### Overview

This lab demonstrates how to easily integrate generative AI features into your applications with just a few lines of code using pgvector, LangChain, and LLMs on Google Cloud.

We will build a sample Python application together that will be able to understand and respond to human language queries about the relational data stored in your PostgreSQL database. In fact, we will further push the creative limits of the application by teaching it to generate new content based on our existing dataset.

This lab utilizes an example of an e-commerce company that operates an online marketplace for buying and selling children's toys. The company aims to incorporate new generative AI experiences into its e-commerce applications for both buyers and sellers on the platform.

The goals are:

- **(Usecase 1) For buyers**: Build a new AI-powered hybrid search, where users can describe their needs in simple English text, along with regular filters (like price, etc.)
- **(Usecase 2) For sellers**: Add a new AI-powered content generation feature, where sellers will get auto-generated item description suggestions for new products that they want to add to the platform.

Dataset: The dataset for this lab has been sampled and created from a larger public retail dataset available at [Kaggle](https://colab.research.google.com/corgiredirector?site=https%3A%2F%2Fwww.kaggle.com%2Fdatasets%2Fpromptcloud%2Fwalmart-product-details-2020). The sampled dataset used in this lab has only about 800 toy products, while the public dataset has over 370,000 products in different categories.

### Objective

At the end of this lab:

- You will have a good understanding of how to use the [pgvector extension](https://github.com/pgvector/pgvector) to store and search vector embeddings in PostgreSQL. Learn more about [vector embeddings](https://cloud.google.com/blog/topics/developers-practitioners/meet-ais-multitool-vector-embeddings).
- You will get a hands-on experience with using the open-source [LangChain framework](https://python.langchain.com/docs/get_started/introduction) to develop applications powered by large language models. LangChain makes it easier to develop and deploy applications against any LLM model in a vendor-agnostic manner.
- You will learn about the powerful features in [Google PaLM models made available through Vertex AI](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/overview).


### Task 1. Install packages

In [ ]:
!pip install numpy pandas
!pip install pgvector
!pip install langchain langchain_google_vertexai transformers
!pip install google-cloud-aiplatform
!pip install psycopg2-binary
!pip install protobuf
!pip install shapely

In [2]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

### Task 2. Download and load the dataset

An AlloyDB cluster named cymbal-alloy-cluster is configured in this lab. To begin, let's locate the AlloyDB cluster's IP address.

1. On the Google Cloud console title bar, type "AlloyDB" in the Search field, then click AlloyDB in the Products & Pages section.
2. Locate the cluster named cymbal-alloy-cluster, and the primary instance named cymbal-instance. The private IP address of this instance serves as your access point for utilizing AlloyDB throughout the lab.

Back in Vertex AI Workbench Notebook, import necessary libraries.

In [1]:
import os
import pandas as pd
import vertexai
from vertexai.language_models import TextEmbeddingModel
from vertexai.generative_models import GenerativeModel
from IPython.display import display, Markdown

from langchain_google_vertexai import VertexAIEmbeddings
import vertexai

PROJECT_ID = "qwiklabs-gcp-02-2cf3961f017f"  # @param {type:"string"}
REGION = "us-central1"  # @param {type:"string"}

# Initialize Vertex AI SDK
vertexai.init(project=PROJECT_ID, location=REGION)

In [2]:
import psycopg2

# Replace with your AlloyDB cluster credentials
cluster_ip_address = "10.103.0.2"
database_user = "postgres"
database_password = "postgres"

# Set environment variables for psql connection
os.environ["PGHOST"] = cluster_ip_address
os.environ["PGUSER"] = database_user
os.environ["PGPASSWORD"] = database_password

# Establish a connection to the database
try:
    conn = psycopg2.connect(
        host=cluster_ip_address,
        user=database_user,
        password=database_password
    )
    print("Connected to the database successfully!")
except Exception as e:
    print("Connection error:", e)
exit(1)

# Read the dataset from the URL
DATASET_URL = "https://github.com/GoogleCloudPlatform/python-docs-samples/raw/main/cloud-sql/postgres/pgvector/data/retail_toy_dataset.csv"
df = pd.read_csv(DATASET_URL)

# Select desired columns and drop missing values
df = df.loc[:, ["product_id", "product_name", "description", "list_price"]]
df = df.dropna()

# Save the DataFrame to the AlloyDB cluster
df.to_sql('products', con=f'postgresql://{cluster_ip_address}', if_exists='replace', index=False)

# Retrieve data from the 'products' table
cur = conn.cursor()
cur.execute("SELECT * FROM products")
results = cur.fetchall()

# Close the connection
conn.close()
print(results[5])

Connected to the database successfully!
('74a695e3675efc2aad11ed73c46db29b', 'Slip N Slide Triple Racer with Slide Boogies', 'Triple Racer Slip and Slide with Boogie Boards. The unit is 16 foot long. The unit has 3 sliding lanes.', 37.21)


### Task 3. Generate Vector Embeddings using a Text Embedding Model

In this section, let's preprocess product descriptions, generate vector embeddings for them, and store the embeddings along with other relevant data in a PostgreSQL database table for downstream analysis or applications.

Run the following code snippet to import the RecursiveTextSplitter class from the LangChain library, which is used for splitting text into smaller chunks. Iterate through each row in the DataFrame df and extract the **product ID** and **description** from each row.

Then, we will split each description into smaller chunks and will create a dictionary for each chunk.

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema import Document

# Set up the text splitter
text_splitter = RecursiveCharacterTextSplitter(
    separators=[".", "\n"],
    chunk_size=500,
    chunk_overlap=0,
    length_function=len,
)

# Define the maximum number of documents to process
max_documents = 50  # Reduced limit to further control API usage
documents = []

# Create Document objects with product_id as metadata
for index, row in df.iterrows():
    product_id = row["product_id"]
    desc = row["description"]
    documents.append(Document(page_content=desc, metadata={"product_id": product_id}))

# Use the text splitter on a subset of documents (e.g., 40-50)
chunked = []
docs = text_splitter.split_documents(documents[40:max_documents])

# Collect split content along with product_id
for doc in docs:
    chunked.append({"product_id": doc.metadata["product_id"], "content": doc.page_content})

print(len(chunked))


34


Run the following code snippet to process product descriptions from a dataset by splitting them into smaller chunks, sending them to Vertex AI for embedding generation, and storing the retrieved embeddings back into the data structure.

In [4]:
from langchain_google_vertexai import VertexAIEmbeddings
from google.cloud import aiplatform
import time

embeddings_service = VertexAIEmbeddings(model_name="textembedding-gecko")

# Helper function to retry failed API requests with exponential backoff.
def retry_with_backoff(func, *args, retry_delay=10, backoff_factor=2.5, **kwargs):  # Increased delay and backoff factor
    max_attempts = 10
    retries = 0
    for i in range(max_attempts):
        try:
            return func(*args, **kwargs)
        except Exception as e:
            print(f"error: {e}")
            retries += 1
            wait = retry_delay * (backoff_factor**retries)
            print(f"Retry after waiting for {wait} seconds...")
            time.sleep(wait)

# Reduced batch size for API calls to manage quota limits
batch_size = 3
for i in range(0, len(chunked), batch_size):
    request = [x["content"] for x in chunked[i : i + batch_size]]
    response = retry_with_backoff(embeddings_service.embed_documents, request)
    # Store the retrieved vector embeddings for each chunk back.
    for x, e in zip(chunked[i : i + batch_size], response):
        x["embedding"] = e

# Store the generated embeddings in a pandas dataframe.
product_embeddings = pd.DataFrame(chunked)
product_embeddings.head()


,product_id,content,embedding
0,8a6d71be41e01b284294ec488508b414,All of our productsWalmartply with internation...,"[-0.0019801377784460783, -0.037444762885570526..."
1,8a6d71be41e01b284294ec488508b414,. Holds Up to 6 Decks Fun for the whole family...,"[0.008327648043632507, -0.03802461177110672, -..."
2,9648838f5badebb9fc0b07f89cc29394,Better circulate water through your pool with ...,"[-0.045468609780073166, -0.0008376826299354434..."
3,9648838f5badebb9fc0b07f89cc29394,".25-inch fitting (11070), 2 strainer grids (11...","[-0.020975708961486816, 0.010413266718387604, ..."
4,9648838f5badebb9fc0b07f89cc29394,. Circulate water through your pool with the h...,"[-0.04218039661645889, -0.014087582007050514, ..."


In [ ]:
!PROJECT_ID=$(gcloud config get-value project) && \
PROJECT_NUMBER=$(gcloud projects list --filter="name=$PROJECT_ID" --format="value(PROJECT_NUMBER)") && \
gcloud projects add-iam-policy-binding $PROJECT_ID \
--member="serviceAccount:service-$PROJECT_NUMBER@gcp-sa-alloydb.iam.gserviceaccount.com" \
--role="roles/aiplatform.user"

Back in the **AlloyDB service page**, click on the cluster named `cymbal-alloy-cluster`, then select **AlloyDB Studio** from the left-hand side menu, enter the following values to sign in, and click on **AUTHENTICATE**.

Enter the following command in the editor to grant the postgres user permission to execute the `embedding` function, install the `google_ml_integration` extension, and generate an embedding for the provided text using the `textembedding-gecko` model. Then, click on Run button at the top.

In [ ]:
GRANT EXECUTE ON FUNCTION embedding TO postgres;

CREATE EXTENSION IF NOT EXISTS google_ml_integration CASCADE;

SELECT embedding('textembedding-gecko', 'AlloyDB is a managed, cloud-hosted SQL database service.');

Once the embeddings are successfully generated, click the Clear button at the top to clear the contents of the editor. Then, run the following query in the editor to prepare the database.

In [ ]:
CREATE EXTENSION IF NOT EXISTS vector;

DROP TABLE IF EXISTS product_embeddings;

CREATE TABLE product_embeddings(
        product_id VARCHAR(1024) NOT NULL PRIMARY KEY,
        content TEXT,
        embedding vector(768)
    );


insert into product_embeddings(product_id, content, embedding)
SELECT
  product_id,
  description as content,
  embedding('textembedding-gecko', description) as embedding
from products
where product_id not in (select product_id from product_embeddings)
limit 10;

### Task 4. Create Indexes for faster Similarity Search

Vector indexes can significantly speed up similarity search operations and avoid the brute-force exact nearest neighbor search that is used by default.

**Pgvector** comes with two types of indexes: `hnsw` and `ivfflat`.

In [ ]:
-- Create an HNSW index on the `product_embeddings` table
CREATE INDEX ON product_embeddings
USING hnsw(embedding vector_cosine_ops)
WITH (m = 24, ef_construction = 100);

-- Create an IVFFLAT index on the `product_embeddings` table
CREATE INDEX ON product_embeddings
USING ivfflat(embedding vector_cosine_ops)
WITH (lists = 100);

In [ ]:
-- conduct similarity search
with e as (
SELECT
    *
FROM
    product_embeddings
ORDER BY
    embedding <-> CAST(embedding('textembedding-gecko','Playing card games') AS vector(768)) asc
LIMIT
    5
)
select
*
from products
where product_id in (select e.product_id from e);

### Task 5. LLMs and LangChain

**Use case 1: Building an AI-curated contextual hybrid search**

Combine natural language query text with regular relational filters to create a powerful hybrid search.


In [6]:
# Please fill in these values.
user_query = "Do you have a toy set that teaches numbers and letters to kids?"  # @param {type:"string"}
min_price = 20  # @param {type:"integer"}
max_price = 100  # @param {type:"integer"}

In [ ]:
qe = embeddings_service.embed_query(user_query)

Use pgvector to find similar products. The pgvector similarity search operators provide powerful semantics to combine the vector search operation with regular query filters in a single SQL query.

In [8]:
import psycopg2
from psycopg2 import sql
from pgvector.psycopg2 import register_vector
import pandas as pd

def main(user_query, min_price, max_price):
    try:
        # AlloyDB cluster connection details (replace with your actual values)
        cluster_ip_address = "10.103.0.2"
        database_user = "postgres"
        database_password = "postgres"

        # Connect to AlloyDB cluster
        conn = psycopg2.connect(
            host=cluster_ip_address,
            user=database_user,
            password=database_password
        )

        # Register the vector type
        register_vector(conn)

        # Get the query embedding
        qe = embeddings_service.embed_query(user_query)

        # Check if qe is valid
        if not qe:
            print("Error: The query embedding is empty.")
            return

        # Perform the similarity search and filtering
        cur = conn.cursor()
        similarity_threshold = 0.5  # Increased threshold for broader matching
        num_matches = 50

        # Modify the SQL query for indexed similarity search
        cur.execute(
            """
            WITH vector_matches AS (
                SELECT product_id, embedding <=> %s::vector AS distance
                FROM product_embeddings
                WHERE embedding <=> %s::vector < %s
                ORDER BY distance ASC
                LIMIT %s
            )
            SELECT product_name, list_price, description
            FROM products
            WHERE product_id IN (SELECT product_id FROM vector_matches)
            AND list_price >= %s AND list_price <= %s
            """,
            (qe, qe, similarity_threshold, num_matches, min_price, max_price)
        )
        results = cur.fetchall()

        # Check if any results are retrieved
        if not results:
            print("No results found. Try adjusting the similarity threshold or checking the data.")
            return

        # Process the results
        matches = []
        for r in results:
            try:
                list_price = round(float(r[1]), 2)  # Attempt conversion and rounding
            except ValueError:
                list_price = r[1]  # Use original value if conversion fails
            matches.append({
                "product_name": r[0],
                "list_price": list_price,
                "description": r[2]
            })

        # Display the results
        matches_df = pd.DataFrame(matches)
        print(matches_df.head(5))

    except Exception as e:
        print(f"Error during database operations: {e}")
    finally:
        # Close the connection
        if conn:
            conn.close()

# Call the main function
main("Do you have a toy set that teaches numbers and letters to kids?", 25, 100)

                                   product_name  list_price  \
0               12"-20" Schwinn Training Wheels       28.17   
1  Slip N Slide Triple Racer with Slide Boogies       37.21   

                                         description  
0  Turn any small bicycle into an instrument for ...  
1  Triple Racer Slip and Slide with Boogie Boards...  


After finding the similar products and their descriptions using pgvector, the next step is to use them for generating a prompt input for the LLM model. Since individual product descriptions can be very long, they may not fit within the specified input payload limit for an LLM model. The `MapReduceChain` from the `LangChain` framework is used to generate and combine short summaries of similarly matched products. The combined summaries are then used to build a high-quality prompt for an input to the LLM model.

In [9]:
from IPython.display import display, Markdown
from langchain_core.documents.base import Document
from langchain.chains.summarize import load_summarize_chain
from langchain_google_vertexai import VertexAI
from langchain_core.prompts import PromptTemplate

# Mock matches data
matches = [
    {"product_name": "Alphabet Learning Toy", "price": 30, "features": "Teaches letters and numbers."},
    {"product_name": "Number Puzzle", "price": 20, "features": "Interactive puzzle for number learning."},
]

# LangChain setup
llm = VertexAI(model_name="gemini-pro")

map_prompt_template = """
            You will be given a detailed description of a toy product.
            This description is enclosed in triple backticks (```).
            Using this description only, extract the name of the toy,
            the price of the toy and its features.

            ```{text}```
            SUMMARY:
            """
map_prompt = PromptTemplate(template=map_prompt_template, input_variables=["text"])

combine_prompt_template = """
                You will be given a detailed description of different toy products
                enclosed in triple backticks (```) and a question enclosed in
                double backticks(``).
                Select one toy that is most relevant to answer the question.
                Using that selected toy description, answer the following
                question in as much detail as possible.
                You should only use the information in the description.
                Your answer should include the name of the toy, the price of the toy
                and its features. Your answer should be less than 200 words.
                Your answer should be in Markdown in a numbered list format.

                Description:
                ```{text}```

                Question:
                ``{user_query}``

                Answer:
                """
combine_prompt = PromptTemplate(
    template=combine_prompt_template, input_variables=["text", "user_query"]
)

# Convert matches to LangChain documents
docs = [
    Document(page_content=f"Name: {match['product_name']}, Price: {match['price']}, Features: {match['features']}")
    for match in matches
]

# Load and invoke the chain
chain = load_summarize_chain(
    llm, chain_type="map_reduce", map_prompt=map_prompt, combine_prompt=combine_prompt
)

# User query
user_query = "Do you have a toy set that teaches numbers and letters to kids?"

# Invoke the chain
output = chain.invoke({
    "input_documents": docs,
    "user_query": user_query,
})

# Extract and display the output
answer = output.get('output_text', ' ')
display(Markdown(answer))


1. **Name:** Number Puzzle
2. **Price:** $20
3. **Features:**
    - Interactive
    - Promotes number learning

Although the Alphabet Learning Toy also teaches letters, the Number Puzzle is a better fit for the question as it specifically focuses on numbers and is more interactive, making it a more engaging learning experience for children. 


**Use case 2: Adding AI-powered creative content generation**

Use knowledge from the existing dataset to generate new AI-powered content from an initial prompt.

In [10]:
# Please fill in these values.
creative_prompt = "A bicycle with brand name 'Roadstar bike' for kids that comes with training wheels and helmet." 

Leverage the pgvector similarity search operator to find an existing product description that closely matches the new product specified in the initial prompt.

In [11]:
import psycopg2
from pgvector.psycopg2 import register_vector

def main():
    try:
        # AlloyDB cluster connection details
        cluster_ip_address = "10.103.0.2"
        database_user = "postgres"
        database_password = "postgres"

        # Connect to AlloyDB cluster
        conn = psycopg2.connect(
            host=cluster_ip_address,
            user=database_user,
            password=database_password
        )

        # Register the vector type
        register_vector(conn)

        # Get the query embedding
        qe = embeddings_service.embed_query(creative_prompt)

        # Check if qe is a valid embedding
        if not qe:
            print("Error: The query embedding is empty.")
            return

        # Set similarity threshold
        similarity_threshold = 0.5
        matches = []

        # Perform the similarity search and filtering
        cur = conn.cursor()
        cur.execute(
            """
            WITH vector_matches AS (
                SELECT product_id, embedding <=> %s::vector AS distance
                FROM product_embeddings
                WHERE embedding <=> %s::vector < %s
                ORDER BY distance ASC
                LIMIT 1
            )
            SELECT description FROM products
            WHERE product_id IN (SELECT product_id FROM vector_matches)
            """,
            (qe, qe, similarity_threshold)
        )

        results = cur.fetchall()

        # Process the results
        for r in results:
            matches.append(r[0])

        if not matches:
            print("No matches found.")
        else:
            print("Matches found:", matches)

    except Exception as e:
        print(f"Error during database operations: {e}")
    finally:
        # Close the connection if it was established
        if conn:
            conn.close()

# Call the main function
main()


Matches found: ['Turn any small bicycle into an instrument for learning to ride with the Schwinn 12"-20" Training Wheels. They feature a slotted design to fit 12" to 20" bikes. The training wheels are easy to assemble, install and remove, so that when your little one is able to ride without assistance, you can take them off. These bicycle training wheels include steel brackets and rubber tires that can stand up to heavy use. Training Wheels, Fits 12 inches - 20 inches bicycles. Est. 1895. Durable Construction: Steel brackets stand up to heavy use. Customizable: Two sets of wheel decals included. Features: Fits Most Childrens Bicycles: Intended for 12 inch - 20 inch bicycles. Steel Brackets: Offer increased durability. Includes two sets of wheel decals: Learn how to ride in style - see images below. Easy to Adjust: Slotted design for size adjustment. Includes: One pair of training wheels, four decals, installation instructions, and all mounting hardware. Tools required: Adjustable wrenc

Use the existing matched product description as the prompt context to generate new creative output from the LLM.

In [12]:
from IPython.display import display, Markdown
from langchain_google_vertexai import VertexAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence

# Define the template
template = """
            You are given descriptions about some similar kind of toys in the context.
            This context is enclosed in triple backticks (```).
            Combine these descriptions and adapt them to match the specifications in
            the initial prompt. All the information from the initial prompt must
            be included. You are allowed to be as creative as possible,
            and describe the new toy in as much detail. Your answer should be
            in markdown in lists and less than 200 words.

            Context:
            ```{context}```

            Initial Prompt:
            {creative_prompt}

            Answer:
        """

prompt = PromptTemplate(
    template=template, input_variables=["context", "creative_prompt"]
)

# Define the LLM
llm = VertexAI(model_name="gemini-pro", temperature=0.7)

# Example `matches` list
matches = [
    {"description": "This is a toy description 1."},
    {"description": "This is a toy description 2."},
    {},  # Missing `description`
    "Invalid item"  # Not a dictionary
]

# Construct the context by extracting valid descriptions
context = "\n".join(
    item["description"] for item in matches if isinstance(item, dict) and "description" in item
)

# Define the creative prompt
creative_prompt = "Describe a toy that is suitable for both indoor and outdoor play."

# Use RunnableSequence for chaining
llm_chain = RunnableSequence(prompt | llm)

# Invoke the chain
answer = llm_chain.invoke({
    "context": context,
    "creative_prompt": creative_prompt,
})

# Display the answer in Markdown format
display(Markdown(answer))



## A 2-in-1 Transforming Playset: Indoor Castle & Outdoor Adventure

This innovative toy seamlessly transforms between a majestic indoor castle and a thrilling outdoor adventure scene, offering endless possibilities for imaginative play. 

**Castle Mode:**

*   **Sturdy Structure:** The castle features a durable, interlocking construction with towers, turrets, and a drawbridge, providing a secure and captivating environment for indoor adventures.
*   **Interactive Features:** A hidden treasure chest, secret passageways, and a working portcullis ignite children's imaginations as they embark on quests, defend the castle, or role-play as their favorite characters.
*   **Cozy Play Space:** The castle's interior transforms into a cozy play space with soft, cushioned walls and a plush rug, perfect for reading, storytelling, or quiet playtime.

**Adventure Mode:**

*   **Transforming Landscape:** With a few simple adjustments, the castle unfolds into a sprawling outdoor scene, complete with a winding path, a hidden cave, and a towering tree.
*   **Nature Exploration:** The set includes realistic accessories like a campfire, a telescope, and binoculars, encouraging children to explore, observe, and connect with nature.
*   **Physical Activity:** The expansive play area provides ample space for running, jumping, and engaging in active play, promoting healthy development and physical fitness.

**Additional Features:**

*   **Durable and Weather-Resistant:** Made from high-quality materials, the toy can withstand both indoor and outdoor use, ensuring long-lasting enjoyment.
*   **Easy to Clean:** The smooth surfaces and removable fabric components allow for quick and effortless cleaning.
*   **Suitable for Multiple Ages:** The versatility of the toy allows children of different ages to enjoy it together, fostering sibling bonding and social interaction.
*   **Imaginative Play:** The 2-in-1 design encourages creativity, problem-solving, and storytelling, fostering a love for imaginative play that extends beyond the toy itself. 
